# RDD Transfer Microbenchmark

In [1]:
import time
from functools import wraps
from typing import Callable, TypeVar, Any

T = TypeVar('T')

def time_function(func: Callable[..., T]) -> Callable[..., T]:
    @wraps(func)
    def wrapper(*args, **kwargs) -> T:
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        
        print(f"{func.__name__} executed in {end_time - start_time:.6f} seconds")
        return result
    
    return wrapper

In [5]:
# IMPORTANT: This code assumes you've done the preparation-related steps in getting-started.ipynb

from ndn_compute_client import NdnComputeClient
client = NdnComputeClient('http://localhost:5214')

In [18]:
import zlib

@time_function
def fetch_urandom():
    random_bytes = client.urandom()
    print(zlib.crc32(random_bytes))

In [19]:
# Pass 1: not cached in NFD
fetch_urandom()

250185802
fetch_urandom executed in 38.774592 seconds


In [20]:
# Pass 2: in NFD content-store
fetch_urandom()

250185802
fetch_urandom executed in 17.186113 seconds


## Results:

Note that I need to restart NFD between trials, so the above code isn't in a loop.



|           | Pass 1 (not cached in NFD) | Pass 2 (in NFD content-store) |
|-----------|----------------------------|-------------------------------|
| 1         | 33.348039                  | 14.590795                     |
| 2         | 33.509490                  | 15.388483                     |
| 3         | 35.397389                  | 16.286589                     |
| 4         | 34.809988                  | 14.662206                     |
| 5         | 38.774592                  | 17.186113                     |
| **Mean**  | 35.17                      | 15.62                         |
| **Stdev** | 2.19                       | 1.11                          |